In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import torch
from kan import KAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm import tqdm

In [ ]:
data = pd.read_csv('file.csv')
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y')
data

In [ ]:
chl_a_values = data[['chl_a']].values
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
data['lof_score'] = lof.fit_predict(chl_a_values)

data = data[data['lof_score'] != -1].drop(columns=['lof_score'])
forecast_steps = 6  
date_range = pd.date_range(start='2002-08-01', end='2024-08-01', freq='MS')
data.set_index('date', inplace=True)
data = data.reindex(date_range)
missing_dates = data[data['chl_a'].isnull()].index





data['chl_a'] = data['chl_a'].fillna(method='ffill')

data['chl_a'] = data['chl_a'].fillna(method='bfill')

stl = STL(data['chl_a'], seasonal=13)
result = stl.fit()

data['chl_a'] = np.where(data['chl_a'].isnull(), result.trend + result.seasonal, data['chl_a'])

data['chl_a'] = data['chl_a'].interpolate(method='linear')
data.to_csv('imputed.csv', index=True)

print(data.isna().sum())


data['month'] = data.index.month

data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)
data = data.drop(['month'], axis=1)
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['chl_a'], label='Chl-a Concentration', color='blue')

imputed_values = data.loc[missing_dates]
plt.scatter(imputed_values.index, imputed_values['chl_a'], color='red', label='Imputed Values')

plt.title('Chlorophyll-a Concentration Over Time')
plt.xlabel('Date')
plt.ylabel('Chlorophyll-a (chl_a)')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
if forecast_steps > 0:
    data_for_forecasting = data.iloc[-forecast_steps:]
    data = data.iloc[:-forecast_steps] 
else:
    data = data
    data_for_forecasting = pd.DataFrame()  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf


plt.figure(figsize=(10, 6))
plot_pacf(data['chl_a'].dropna(), lags=12, method='ywm')  plt.title('Partial Autocorrelation of Chlorophyll-a')
plt.show()


In [ ]:
data

In [ ]:
data_LSTM = data.copy()
data_LSTM

In [ ]:
def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    
    r = numerator / denominator
    return r ** 2

In [ ]:
import pandas as pd

lag = 12  for i in range(1, lag + 1):
    data[f'yt-{i}'] = data['chl_a'].shift(i)

data.dropna(inplace=True)


In [ ]:
data_LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


X = data[[f'yt-{i}' for i in range(1, lag + 1)]].values
y = data['chl_a'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

best_r2 = -np.inf
best_model_metrics = None
best_train_results_df = None
best_test_results_df = None

for run in range(20):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=run)

        model = LinearRegression()

        model.fit(X_train, y_train)

        y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

        train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)

        train_custom_r2 = custom_r2(y_train, y_pred_train)
    test_custom_r2 = custom_r2(y_test, y_pred_test)

        print(f"Run {run + 1}: Train R2 = {train_r2:.4f}, Test R2 = {test_r2:.4f}, Custom Test R2 = {test_custom_r2:.4f}")

        if test_custom_r2 > best_r2:
        best_r2 = test_custom_r2

                train_mse = mean_squared_error(y_train, y_pred_train)
        test_mse = mean_squared_error(y_test, y_pred_test)

        train_mae = mean_absolute_error(y_train, y_pred_train)
        train_rmse = np.sqrt(train_mse)

        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_rmse = np.sqrt(test_mse)

                best_train_results_df = pd.DataFrame({
            'Actual_Train': y_train.flatten(),
            'Predicted_Train': y_pred_train.flatten()
        })

        best_test_results_df = pd.DataFrame({
            'Actual_Test': y_test.flatten(),
            'Predicted_Test': y_pred_test.flatten()
        })

                best_model_metrics = {
            'Train MAE': train_mae,
            'Train MSE': train_mse,
            'Train RMSE': train_rmse,
            'Train R2': train_r2,
            'Test MAE': test_mae,
            'Test MSE': test_mse,
            'Test RMSE': test_rmse,
            'Test R2': test_r2
        }
        
Act_Train = y_train.flatten()
Act_Test = y_test.flatten()
MLR_Train = y_pred_train.flatten()
MLR_Test = y_pred_test.flatten()

with pd.ExcelWriter('MLR.xlsx') as writer:
    best_train_results_df.to_excel(writer, sheet_name='Train', index=False)
    best_test_results_df.to_excel(writer, sheet_name='Test', index=False)

        metrics_df = pd.DataFrame(best_model_metrics, index=[0])
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

print(f"The best model has been exported to 'MLR_South_Caspian_Best.xlsx' with a custom R2 of {best_r2:.4f}")


# ANN

### Hyperparameter Optimization

In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class ANNModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(ANNModel, self).__init__()
        layers = []
        current_size = input_size

                for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))              current_size = hidden_size

        layers.append(nn.Linear(current_size, output_size))          self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def train_and_evaluate_ann(hidden_sizes, learning_rate, dropout_rate):
    model = ANNModel(input_size=5, hidden_sizes=hidden_sizes, output_size=1, dropout_rate=dropout_rate)

        criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

                outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

                loss.backward()
        optimizer.step()

        model.eval()
    with torch.no_grad():
        predictions_test = model(X_test_tensor).numpy()
        actual_test = y_test_tensor.numpy()

        test_r2 = np.corrcoef(actual_test.flatten(), predictions_test.flatten())[0, 1] ** 2
    return test_r2

learning_rates = [0.001, 0.01, 0.1]
hidden_layer_configs = [[32], [64], [32, 64], [64, 128]]  dropout_rates = [0.1, 0.2, 0.3]

best_r2 = -float('inf')
best_params = None

for lr in learning_rates:
    for hidden_sizes in hidden_layer_configs:
        for dropout_rate in dropout_rates:
            print(f"Training with LR={lr}, Hidden Sizes={hidden_sizes}, Dropout Rate={dropout_rate}")
            test_r2 = train_and_evaluate_ann(hidden_sizes, lr, dropout_rate)
            print(f"Test R2: {test_r2:.4f}")

            if test_r2 > best_r2:
                best_r2 = test_r2
                best_params = {'Learning Rate': lr, 'Hidden Sizes': hidden_sizes, 'Dropout Rate': dropout_rate}

print(f"Best Hyperparameters: {best_params} with Test R2: {best_r2:.4f}")



### Main Code

In [ ]:
import pandas as pd
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

X = data[[f'yt-{i}' for i in range(1, lag + 1)] + ['month_sin', 'month_cos']].values
y = data['chl_a'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(14, 32)          self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 64)          self.relu = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

def train_and_evaluate_ann():
    model = ANNModel()

        criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()

                outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)

                loss.backward()
        optimizer.step()

        model.eval()
    with torch.no_grad():
        predictions_train = model(X_train_tensor).numpy()
        predictions_test = model(X_test_tensor).numpy()
        actual_train = y_train_tensor.numpy()
        actual_test = y_test_tensor.numpy()

        train_r2 = np.corrcoef(actual_train.flatten(), predictions_train.flatten())[0, 1] ** 2
    test_r2 = np.corrcoef(actual_test.flatten(), predictions_test.flatten())[0, 1] ** 2

        train_custom_r2 = custom_r2(actual_train, predictions_train)
    test_custom_r2 = custom_r2(actual_test, predictions_test)

        train_corr, train_p_value = pearsonr(actual_train.flatten(), predictions_train.flatten())
    test_corr, test_p_value = pearsonr(actual_test.flatten(), predictions_test.flatten())

    return (model, train_custom_r2, test_custom_r2, train_r2, test_r2,
            train_p_value, test_p_value, predictions_train, actual_train,
            predictions_test, actual_test)

best_r2 = -float('inf')
best_model = None
best_metrics = None

for i in range(20):
    (model, train_custom_r2, test_custom_r2, train_r2, test_r2, train_p_value, 
     test_p_value, predictions_train, actual_train, predictions_test, actual_test) = train_and_evaluate_ann()

    if test_custom_r2 > best_r2:
        best_r2 = test_custom_r2
        best_model = model
        best_metrics = {
            'Train Custom R2': train_custom_r2,
            'Test Custom R2': test_custom_r2,
            'Train R2': train_r2,
            'Test R2': test_r2,
            'Train p-value': train_p_value,
            'Test p-value': test_p_value,
            'Train MAE': np.mean(np.abs(predictions_train - actual_train)),
            'Test MAE': np.mean(np.abs(predictions_test - actual_test)),
            'Train MSE': np.mean((predictions_train - actual_train) ** 2),
            'Test MSE': np.mean((predictions_test - actual_test) ** 2),
            'Train RMSE': np.sqrt(np.mean((predictions_train - actual_train) ** 2)),
            'Test RMSE': np.sqrt(np.mean((predictions_test - actual_test) ** 2)),
            'Train Predictions': predictions_train.flatten(),
            'Test Predictions': predictions_test.flatten(),
            'Train Actual': actual_train.flatten(),
            'Test Actual': actual_test.flatten()
        }

    print(f"Run {i+1}/20 - Train Custom R2: {train_custom_r2:.4f} - Test Custom R2: {test_custom_r2:.4f}")

best_train_results_df = pd.DataFrame({
    'Actual_Train': best_metrics['Train Actual'],
    'Predicted_Train': best_metrics['Train Predictions']
})

best_test_results_df = pd.DataFrame({
    'Actual_Test': best_metrics['Test Actual'],
    'Predicted_Test': best_metrics['Test Predictions']
})

metrics_df = pd.DataFrame({
    'Metric': ['Custom R2', 'Traditional R2', 'p-value', 'MAE', 'MSE', 'RMSE'],
    'Train': [best_metrics['Train Custom R2'], best_metrics['Train R2'], best_metrics['Train p-value'], 
              best_metrics['Train MAE'], best_metrics['Train MSE'], best_metrics['Train RMSE']],
    'Test': [best_metrics['Test Custom R2'], best_metrics['Test R2'], best_metrics['Test p-value'],
             best_metrics['Test MAE'], best_metrics['Test MSE'], best_metrics['Test RMSE']]
})


ANN_Train = best_metrics['Train Predictions']
ANN_Test = best_metrics['Test Predictions']
with pd.ExcelWriter('ANN.xlsx') as writer:
    best_train_results_df.to_excel(writer, sheet_name='Train', index=False)
    best_test_results_df.to_excel(writer, sheet_name='Test', index=False)
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

print(f"Best model's Test Custom R2: {best_metrics['Test Custom R2']:.4f}")
print("Predicted vs Actual values and metrics for the best model have been exported to ANN_South_Best.xlsx")


### ANN Forecast

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input(last_lag_values, forecast_dates, i):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values, month_sin, month_cos])

    return forecast_input

def forecast_next_records(model, data, lag=lag, forecast_steps=1):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input = prepare_forecast_input(last_lag_values, forecast_dates, i)

                forecast_input_tensor = torch.tensor(forecast_input, dtype=torch.float32).unsqueeze(0)

                model.eval()
        with torch.no_grad():
            forecast_value_scaled = model(forecast_input_tensor).cpu().numpy().flatten()[0]

                scaler_y = StandardScaler()
        scaler_y.fit(data[['chl_a']].values)
        forecast_value = scaler_y.inverse_transform(np.array([[forecast_value_scaled]])).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates

ANN_forecast_values, forecast_dates = forecast_next_records(best_model, data, lag=lag, forecast_steps=forecast_steps)

for date, value in zip(forecast_dates, ANN_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


# KAN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from deepkan import SplineLinearLayer

class KANTimeSeries(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, num_knots=5, spline_order=3,
                 noise_scale=0.1, base_scale=0.1, spline_scale=1.0,
                 activation=nn.SiLU, grid_epsilon=1, grid_range=[-1, 1]):
        super(KANTimeSeries, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        self.layers = nn.ModuleList()
        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.layers.append(SplineLinearLayer(prev_size, hidden_size, num_knots, spline_order,
                                                 noise_scale, base_scale, spline_scale,
                                                 activation, grid_epsilon, grid_range))
            prev_size = hidden_size

        self.output_layer = SplineLinearLayer(prev_size, output_size, num_knots, spline_order,
                                              noise_scale, base_scale, spline_scale,
                                              activation, grid_epsilon, grid_range)

    def forward(self, x, update_knots=False):
        for layer in self.layers:
            if update_knots:
                layer._update_knots(x)
            x = layer(x)

        if update_knots:
            self.output_layer._update_knots(x)
        x = self.output_layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        loss = 0
        for layer in self.layers:
            loss += layer._regularization_loss(regularize_activation, regularize_entropy)
        loss += self.output_layer._regularization_loss(regularize_activation, regularize_entropy)
        return loss

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        loss = 0
        for layer in self.layers:
            loss += layer._regularization_loss(regularize_activation, regularize_entropy)
        loss += self.output_layer._regularization_loss(regularize_activation, regularize_entropy)
        return loss


In [ ]:

dat_KAN = data_LSTM[['chl_a', 'month_sin', 'month_cos']].values.astype(np.float32)

scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(dat_KAN)

def create_sequences(dat_KAN, window_size):
    sequences = []
    for i in range(len(dat_KAN) - window_size):
        input_seq = dat_KAN[i:i+window_size]          output = dat_KAN[i+window_size, 0]          sequences.append((input_seq, output))
    return sequences

window_size = 12  sequences = create_sequences(data_normalized, window_size)

inputs_torch = torch.tensor([seq[0] for seq in sequences], dtype=torch.float32)
targets_torch = torch.tensor([seq[1] for seq in sequences], dtype=torch.float32)

split_idx = int(len(inputs_torch) * 0.8)
train_inputs_torch = inputs_torch[:split_idx]
test_inputs_torch = inputs_torch[split_idx:]
train_targets_torch = targets_torch[:split_idx]
test_targets_torch = targets_torch[split_idx:]

input_size = window_size * 3  hidden_sizes = [32]
num_knots = 5
spline_order = 3
model_torch = KANTimeSeries(input_size, hidden_sizes, 1, num_knots, spline_order)

optimizer_torch = torch.optim.Adam(model_torch.parameters(), lr=0.01)
criterion_torch = torch.nn.MSELoss()

epochs = 110
for epoch in range(epochs):
    model_torch.train()
    epoch_loss = 0
    for i in range(len(train_inputs_torch)):
        optimizer_torch.zero_grad()
        output = model_torch(train_inputs_torch[i:i+1].view(1, -1))          target = train_targets_torch[i].view_as(output)
        loss = criterion_torch(output, target)
        loss.backward()
        optimizer_torch.step()
        epoch_loss += loss.item()
    print(f'KAN: Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_inputs_torch):.6f}')

model_torch.eval()
with torch.no_grad():
    train_pred_torch = model_torch(train_inputs_torch.view(len(train_inputs_torch), -1)).squeeze().numpy()
    test_pred_torch = model_torch(test_inputs_torch.view(len(test_inputs_torch), -1)).squeeze().numpy()

train_pred_denorm_torch = scaler.inverse_transform(np.hstack([train_pred_torch.reshape(-1, 1), np.zeros((len(train_pred_torch), 2))]))[:, 0]
test_pred_denorm_torch = scaler.inverse_transform(np.hstack([test_pred_torch.reshape(-1, 1), np.zeros((len(test_pred_torch), 2))]))[:, 0]
train_actual_denorm_torch = scaler.inverse_transform(np.hstack([train_targets_torch.numpy().reshape(-1, 1), np.zeros((len(train_targets_torch), 2))]))[:, 0]
test_actual_denorm_torch = scaler.inverse_transform(np.hstack([test_targets_torch.numpy().reshape(-1, 1), np.zeros((len(test_targets_torch), 2))]))[:, 0]

model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(window_size * 3,)),
    tf.keras.layers.Dense(1)
])

model_tf.compile(optimizer='adam', loss='mean_squared_error')
history_tf = model_tf.fit(train_inputs_torch.numpy().reshape(len(train_inputs_torch), -1), train_targets_torch.numpy(), epochs=epochs, verbose=1)

train_pred_tf = model_tf.predict(train_inputs_torch.numpy().reshape(len(train_inputs_torch), -1)).flatten()
test_pred_tf = model_tf.predict(test_inputs_torch.numpy().reshape(len(test_inputs_torch), -1)).flatten()

train_pred_denorm_tf = scaler.inverse_transform(np.hstack([train_pred_tf.reshape(-1, 1), np.zeros((len(train_pred_tf), 2))]))[:, 0]
test_pred_denorm_tf = scaler.inverse_transform(np.hstack([test_pred_tf.reshape(-1, 1), np.zeros((len(test_pred_tf), 2))]))[:, 0]

train_actual_denorm_tf = scaler.inverse_transform(np.hstack([train_targets_torch.numpy().reshape(-1, 1), np.zeros((len(train_targets_torch), 2))]))[:, 0]
test_actual_denorm_tf = scaler.inverse_transform(np.hstack([test_targets_torch.numpy().reshape(-1, 1), np.zeros((len(test_targets_torch), 2))]))[:, 0]

train_results_df = pd.DataFrame({
    'Actual': train_actual_denorm_torch,
    'KAN_Predicted': train_pred_denorm_torch,
    'MLP_Predicted': train_pred_denorm_tf
})

test_results_df = pd.DataFrame({
    'Actual': test_actual_denorm_torch,
    'KAN_Predicted': test_pred_denorm_torch,
    'MLP_Predicted': test_pred_denorm_tf
})

print("Training Results:")
print(train_results_df)

print("Test Results:")
print(test_results_df)


In [ ]:
import datetime

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def forecast_next_steps(model, last_window, n_steps, window_size, start_date):
    forecasts = []
    current_window = last_window.copy()      current_date = start_date

    for _ in range(n_steps):
                current_window_torch = torch.tensor(current_window.reshape(1, -1), dtype=torch.float32)
        
                with torch.no_grad():
            next_pred = model(current_window_torch).item()  
                forecasts.append(next_pred)
        
                        current_date += datetime.timedelta(days=30)          month_sin, month_cos = calculate_month_cyclic_features(current_date)
        
                next_input = np.array([next_pred, month_sin, month_cos])          current_window = np.vstack((current_window[1:], next_input))          
    return forecasts

last_window = data_normalized[-window_size:]  n_steps = 6
start_date = datetime.date(2024, 9, 1) 

next_6_predictions_torch = forecast_next_steps(model_torch, last_window, n_steps, window_size, start_date)

next_6_predictions_denorm_torch = scaler.inverse_transform(np.hstack([np.array(next_6_predictions_torch).reshape(-1, 1), np.zeros((len(next_6_predictions_torch), 2))]))[:, 0]

print("Forecasted next 6 chl_a values using KAN model:")
print(next_6_predictions_denorm_torch)

def forecast_next_steps_tf(model, last_window, n_steps, window_size, start_date):
    forecasts = []
    current_window = last_window.copy()      current_date = start_date

    for _ in range(n_steps):
                current_window_tf = current_window.reshape(1, -1)
        
                next_pred = model.predict(current_window_tf).item()          
                forecasts.append(next_pred)
        
                        current_date += datetime.timedelta(days=30)          month_sin, month_cos = calculate_month_cyclic_features(current_date)
        
                next_input = np.array([next_pred, month_sin, month_cos])          current_window = np.vstack((current_window[1:], next_input))          
    return forecasts

next_6_predictions_tf = forecast_next_steps_tf(model_tf, last_window, n_steps, window_size, start_date)

next_6_predictions_denorm_tf = scaler.inverse_transform(np.hstack([np.array(next_6_predictions_tf).reshape(-1, 1), np.zeros((len(next_6_predictions_tf), 2))]))[:, 0]

print("Forecasted next 6 chl_a values using MLP model:")
print(next_6_predictions_denorm_tf)


In [ ]:
import numpy as np
import torch
from kan import KAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

def load_chla_dataset(data, lag):
    target = data['chl_a'].values
        input_data = data[[f'yt-{i}' for i in range(1, lag + 1)]]

        scaler = StandardScaler()
    input_data = scaler.fit_transform(input_data)

        input_tensor = torch.tensor(input_data, dtype=torch.float32, device=device)
    target_tensor = torch.tensor(target, dtype=torch.float32, device=device).reshape(-1, 1)

        train_data, test_data, train_target, test_target = train_test_split(input_tensor, target_tensor, test_size=0.2, shuffle=False, random_state=42)

        return {
        'train_input': train_data,
        'train_label': train_target,
        'test_input': test_data,
        'test_label': test_target
    }


def build_and_train_kan_model(dataset):
    KAN_model = KAN(
                width=[dataset['train_input'].shape[1], 64, 1],
        grid=3,
        k=2,
        noise_scale=0.1,
        noise_scale_base=0.1,
        symbolic_enabled=False,
        bias_trainable=False,
        grid_eps=0.02,
        grid_range=[-1, 1],
        sp_trainable=True,
        sb_trainable=True,
        device=device
    )

        def train_mse():
        with torch.no_grad():
            predictions = KAN_model(dataset['train_input'])
            return torch.nn.functional.mse_loss(predictions, dataset['train_label'])

    def test_mse():
        with torch.no_grad():
            predictions = KAN_model(dataset['test_input'])
            return torch.nn.functional.mse_loss(predictions, dataset['test_label'])

        results = KAN_model.train(
        dataset, opt="LBFGS", device=device, metrics=(train_mse, test_mse),
        steps=10,
        log=1,
        lamb=0.01,
        lamb_l1=1,
        lamb_entropy=2,
        lamb_coef=0,
        lamb_coefdiff=0,
        update_grid=True,
        grid_update_num=10,
        loss_fn=torch.nn.SmoothL1Loss(),
        lr=0.01,
        stop_grid_update_step=50,
        batch=-1,
        small_mag_threshold=1e-16,
        small_reg_factor=0.1,
        sglr_avoid=False
    )

        with torch.no_grad():
        y_pred_train = KAN_model(dataset['train_input']).cpu().numpy().flatten()
        y_pred_test = KAN_model(dataset['test_input']).cpu().numpy().flatten()

        train_metrics = {
        'R2': r2_score(dataset['train_label'].cpu().numpy(), y_pred_train),
        'MAE': mean_absolute_error(dataset['train_label'].cpu().numpy(), y_pred_train),
        'MSE': mean_squared_error(dataset['train_label'].cpu().numpy(), y_pred_train),
        'RMSE': np.sqrt(mean_squared_error(dataset['train_label'].cpu().numpy(), y_pred_train)),
        'R': np.corrcoef(dataset['train_label'].cpu().numpy().flatten(), y_pred_train)[0, 1]
    }

    test_metrics = {
        'R2': r2_score(dataset['test_label'].cpu().numpy(), y_pred_test),
        'MAE': mean_absolute_error(dataset['test_label'].cpu().numpy(), y_pred_test),
        'MSE': mean_squared_error(dataset['test_label'].cpu().numpy(), y_pred_test),
        'RMSE': np.sqrt(mean_squared_error(dataset['test_label'].cpu().numpy(), y_pred_test)),
        'R': np.corrcoef(dataset['test_label'].cpu().numpy().flatten(), y_pred_test)[0, 1]
    }

    return KAN_model, train_metrics, test_metrics, y_pred_train, y_pred_test


metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {}
best_y_preds_train = {}


lag=lag
chla_dataset = load_chla_dataset(data, lag=lag)

KAN_model, train_metrics, test_metrics, y_pred_train, y_pred_test = build_and_train_kan_model(chla_dataset)

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([train_metrics])], ignore_index=True)

best_y_preds_test['Actual'] = chla_dataset['test_label'].cpu().numpy().flatten()
best_y_preds_test['Predicted'] = y_pred_test

best_y_preds_train['Actual'] = chla_dataset['train_label'].cpu().numpy().flatten()
best_y_preds_train['Predicted'] = y_pred_train

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': train_metrics['R2'], 'R': train_metrics['R'], 
                  'MSE': train_metrics['MSE'], 'RMSE': train_metrics['RMSE'], 'MAE': train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': test_metrics['R2'], 'R': test_metrics['R'], 
                  'MSE': test_metrics['MSE'], 'RMSE': test_metrics['RMSE'], 'MAE': test_metrics['MAE']}, index=[1])
])

KAN_Train = y_pred_train
KAN_Test = y_pred_test
with pd.ExcelWriter('KAN.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'KAN_South.xlsx'")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input(data, last_lag_values, forecast_dates, i):
        
            forecast_input = np.hstack([last_lag_values])

        return forecast_input

def forecast_next_records(KAN_model, data, lag=lag, forecast_steps=forecast_steps, device='cpu'):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []
    for i in range(forecast_steps):
                forecast_input = prepare_forecast_input(data, last_lag_values, forecast_dates, i)
        
                forecast_input_tensor = torch.tensor(forecast_input, dtype=torch.float32, device=device).unsqueeze(0)
        print(forecast_input_tensor)
                KAN_model.to(device)
        
                with torch.no_grad():
            forecast_value_scaled = KAN_model(forecast_input_tensor).cpu().numpy().flatten()[0]
        
                scaler_y = StandardScaler()
        scaler_y.fit(data[['chl_a']].values)
        forecast_value = scaler_y.inverse_transform(np.array([[forecast_value_scaled]])).flatten()[0]
        
                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value          
    
    return forecast_values, forecast_dates

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

KAN_forecast_values, forecast_dates = forecast_next_records(KAN_model, data, lag=lag, forecast_steps=forecast_steps, device=device)

for date, value in zip(forecast_dates, KAN_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


In [ ]:
import pandas as pd


forecast_data = {
    'Date': forecast_dates,      'Actual': data_for_forecasting['chl_a'].values,      'ANN_Forecast': ANN_forecast_values,
    'KAN_Forecast': KAN_forecast_values,
    'RF_Forecast': RF_forecast_values,
    'GPR_Forecast': GPR_forecast_values,
    'SVR_Forecast': SVR_forecast_values
}

forecast_df = pd.DataFrame(forecast_data)

forecast_df.to_excel('forecast.xlsx', index=False)

print("Forecast results have been successfully exported to 'forecast.xlsx'")


# LSTM

### Hyperparameter Tunning

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]          out = self.fc(lstm_out)
        return out

def prepare_data_with_time_steps(data_LSTM, time_steps=lag):
        chl_a_values = data_LSTM['chl_a'].values
    month_sin = data_LSTM['month_sin'].values
    month_cos = data_LSTM['month_cos'].values

    X, y = [], []
    for i in range(len(chl_a_values) - time_steps):
                X.append(
               np.hstack([chl_a_values[i:i + time_steps].reshape(-1, 1), 
               np.tile([month_sin[i + time_steps], month_cos[i + time_steps]], (time_steps, 1))])
        )

        y.append(chl_a_values[i + time_steps])  
    X, y = np.array(X), np.array(y).reshape(-1, 1)
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
        X = X.reshape(X.shape[0], -1)      X = scaler_X.fit_transform(X)      X = X.reshape(X.shape[0], time_steps, -1)  
        y = scaler_y.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, scaler_y

def train_and_evaluate_lstm(X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_layers, dropout_rate, lr, num_epochs=100):
    lstm_model = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers, dropout_rate=dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(lstm_model.parameters(), lr=lr)
    
        for epoch in range(num_epochs):
        lstm_model.train()
        optimizer.zero_grad()
        output = lstm_model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
    
        lstm_model.eval()
    with torch.no_grad():
        y_test_pred = lstm_model(X_test).cpu().numpy()
    
        r2 = r2_score(y_test.cpu().numpy(), y_test_pred)
    return r2, lstm_model


time_steps = lag
input_size = 3  output_size = 1

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_time_steps(data_LSTM, time_steps=time_steps)

param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_layers': [1, 2, 3],
    'hidden_size': [16, 32, 64],
    'dropout_rate': [0.1, 0.2, 0.3]
}

best_r2 = -float('inf')
best_params = None
best_model = None

for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    r2, lstm_model = train_and_evaluate_lstm(
        X_train, y_train, X_test, y_test, 
        input_size=input_size, 
        hidden_size=params['hidden_size'], 
        output_size=output_size,
        num_layers=params['num_layers'], 
        dropout_rate=params['dropout_rate'], 
        lr=params['learning_rate'],
        num_epochs=100      )

    if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_model = lstm_model

print(f"Best parameters: {best_params}")
print(f"Best R2 score: {best_r2}")




In [ ]:
best_params

### Main Code

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=0)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]          out = self.fc(lstm_out)
        return out

def prepare_data_with_time_steps(data_LSTM, time_steps=lag):
        chl_a_values = data_LSTM['chl_a'].values
    month_sin = data_LSTM['month_sin'].values
    month_cos = data_LSTM['month_cos'].values

    X, y = [], []
    for i in range(len(chl_a_values) - time_steps):
                X.append(
               np.hstack([chl_a_values[i:i + time_steps].reshape(-1, 1), 
               np.tile([month_sin[i + time_steps], month_cos[i + time_steps]], (time_steps, 1))])
        )

        y.append(chl_a_values[i + time_steps])  
    X, y = np.array(X), np.array(y).reshape(-1, 1)
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
        X = X.reshape(X.shape[0], -1)      X = scaler_X.fit_transform(X)      X = X.reshape(X.shape[0], time_steps, -1)  
        y = scaler_y.fit_transform(y)

    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, scaler_y

def train_and_evaluate_lstm(X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=100, lr=0.01):
    lstm_model = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)
    
        for epoch in range(num_epochs):
        lstm_model.train()
        optimizer.zero_grad()
        output = lstm_model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        lstm_model.eval()
    with torch.no_grad():
        y_train_pred = lstm_model(X_train).cpu().numpy()
        y_test_pred = lstm_model(X_test).cpu().numpy()

        train_metrics = {
        'R2': r2_score(y_train.cpu().numpy(), y_train_pred),
        'MAE': mean_absolute_error(y_train.cpu().numpy(), y_train_pred),
        'MSE': mean_squared_error(y_train.cpu().numpy(), y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train.cpu().numpy(), y_train_pred)),
        'R': np.corrcoef(y_train.cpu().numpy().flatten(), y_train_pred.flatten())[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test.cpu().numpy(), y_test_pred),
        'MAE': mean_absolute_error(y_test.cpu().numpy(), y_test_pred),
        'MSE': mean_squared_error(y_test.cpu().numpy(), y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test.cpu().numpy(), y_test_pred)),
        'R': np.corrcoef(y_test.cpu().numpy().flatten(), y_test_pred.flatten())[0, 1]
    }

        return train_metrics, test_metrics, y_train_pred, y_test_pred, lstm_model


best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_lstm_model = None  
time_steps = lag
input_size = 3  hidden_size = 32
output_size = 1

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_time_steps(data_LSTM, time_steps=time_steps)



for i in range(5):
    print(f"Run {i+1}/20")
    train_metrics, test_metrics, y_train_pred, y_test_pred, lstm_model = train_and_evaluate_lstm(
        X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=100, lr=0.01
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_lstm_model = lstm_model  


best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred)
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred)
y_train_actual = scaler_y.inverse_transform(y_train.cpu().numpy())
y_test_actual = scaler_y.inverse_transform(y_test.cpu().numpy())

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {'Actual': y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])

LSTM_Train = best_y_train_pred.flatten()
LSTM_Test = best_y_test_pred.flatten()
with pd.ExcelWriter('LSTM.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'LSTM_South_Best_R2.xlsx'")


### LSTM Forecast

In [ ]:
import numpy as np
import pandas as pd
import torch

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_lstm(last_lag_values, forecast_dates, i, lag=lag, device='cpu'):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values.reshape(-1, 1), np.tile([month_sin, month_cos], (lag, 1))])
    
        forecast_input_tensor = torch.tensor(forecast_input, dtype=torch.float32, device=device).unsqueeze(0)  
    return forecast_input_tensor

def forecast_next_records_lstm(lstm_model, data, lag=lag, forecast_steps=1, device='cpu'):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input_tensor = prepare_forecast_input_lstm(last_lag_values, forecast_dates, i, lag=lag, device=device)

                lstm_model.eval()
        with torch.no_grad():
            forecast_value_scaled = lstm_model(forecast_input_tensor).cpu().numpy().flatten()[0]

                scaler_y = StandardScaler()
        scaler_y.fit(data[['chl_a']].values)
        forecast_value = scaler_y.inverse_transform(np.array([[forecast_value_scaled]])).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LSTM_forecast_values, forecast_dates = forecast_next_records_lstm(best_lstm_model, data_LSTM, lag=lag, forecast_steps=forecast_steps, device=device)

for date, value in zip(forecast_dates, LSTM_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


# GRU

### Hyperparameter Tunning

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_rate):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:, -1, :]          out = self.fc(gru_out)
        return out

def train_and_evaluate_gru(X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_layers, dropout_rate, lr, num_epochs=100):
    gru_model = GRUModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers, dropout_rate=dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(gru_model.parameters(), lr=lr)
    
        for epoch in range(num_epochs):
        gru_model.train()
        optimizer.zero_grad()
        output = gru_model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
    
        gru_model.eval()
    with torch.no_grad():
        y_test_pred = gru_model(X_test).cpu().numpy()

        r2 = r2_score(y_test.cpu().numpy(), y_test_pred)
    return r2, gru_model

time_steps = lag
input_size = 3  output_size = 1

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_time_steps(data_LSTM, time_steps=time_steps)

param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_layers': [1, 2, 3],
    'hidden_size': [16, 32, 64],
    'dropout_rate': [0.1, 0.2, 0.3]
}

best_r2 = -float('inf')
best_params = None
best_model = None

for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    r2, gru_model = train_and_evaluate_gru(
        X_train, y_train, X_test, y_test, 
        input_size=input_size, 
        hidden_size=params['hidden_size'], 
        output_size=output_size,
        num_layers=params['num_layers'], 
        dropout_rate=params['dropout_rate'], 
        lr=params['learning_rate'],
        num_epochs=100      )

        if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_model = gru_model

print(f"Best parameters: {best_params}")
print(f"Best R2 score: {best_r2}")



### Main Code

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:, -1, :]          out = self.fc(gru_out)
        return out

def prepare_data_with_time_steps(data_lstm, time_steps=lag):
    chl_a_values = data_lstm['chl_a'].values
    month_sin = data_lstm['month_sin'].values
    month_cos = data_lstm['month_cos'].values

    X, y = [], []
    for i in range(len(chl_a_values) - time_steps):
        X.append(
               np.hstack([chl_a_values[i:i + time_steps].reshape(-1, 1), 
               np.tile([month_sin[i + time_steps], month_cos[i + time_steps]], (time_steps, 1))])
        )
        y.append(chl_a_values[i + time_steps])

    X, y = np.array(X), np.array(y).reshape(-1, 1)
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X = X.reshape(X.shape[0], -1)      X = scaler_X.fit_transform(X)
    X = X.reshape(X.shape[0], time_steps, -1)  
        y = scaler_y.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, scaler_y

def train_and_evaluate_gru(X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=100, lr=0.001):
    gru_model = GRUModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(gru_model.parameters(), lr=lr)
    
        for epoch in range(num_epochs):
        gru_model.train()
        optimizer.zero_grad()
        output = gru_model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        gru_model.eval()
    with torch.no_grad():
        y_train_pred = gru_model(X_train).cpu().numpy()
        y_test_pred = gru_model(X_test).cpu().numpy()

        train_metrics = {
        'R2': r2_score(y_train.cpu().numpy(), y_train_pred),
        'MAE': mean_absolute_error(y_train.cpu().numpy(), y_train_pred),
        'MSE': mean_squared_error(y_train.cpu().numpy(), y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train.cpu().numpy(), y_train_pred)),
        'R': np.corrcoef(y_train.cpu().numpy().flatten(), y_train_pred.flatten())[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test.cpu().numpy(), y_test_pred),
        'MAE': mean_absolute_error(y_test.cpu().numpy(), y_test_pred),
        'MSE': mean_squared_error(y_test.cpu().numpy(), y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test.cpu().numpy(), y_test_pred)),
        'R': np.corrcoef(y_test.cpu().numpy().flatten(), y_test_pred.flatten())[0, 1]
    }

        return train_metrics, test_metrics, y_train_pred, y_test_pred, gru_model


best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_gru_model = None  
time_steps = lag
input_size = 3  hidden_size = 32
output_size = 1

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_time_steps(data_LSTM, time_steps=time_steps)

for i in range(20):
    print(f"Run {i+1}/20")
    train_metrics, test_metrics, y_train_pred, y_test_pred, gru_model = train_and_evaluate_gru(
        X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=500, lr=0.001
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_gru_model = gru_model  

best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred)
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred)
y_train_actual = scaler_y.inverse_transform(y_train.cpu().numpy())
y_test_actual = scaler_y.inverse_transform(y_test.cpu().numpy())

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {'Actual': y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])

with pd.ExcelWriter('GRU.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'Best_GRU_South_TimeSteps.xlsx'")


### GRU Forecast

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_gru(last_lag_values, forecast_dates, i, lag=lag, device='cpu'):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values.reshape(-1, 1), np.tile([month_sin, month_cos], (lag, 1))])
    
        forecast_input_tensor = torch.tensor(forecast_input, dtype=torch.float32, device=device).unsqueeze(0)  
    return forecast_input_tensor

def forecast_next_records_gru(gru_model, data, lag=lag, forecast_steps=1, device='cpu'):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input_tensor = prepare_forecast_input_gru(last_lag_values, forecast_dates, i, lag=lag, device=device)

                gru_model.eval()
        with torch.no_grad():
            forecast_value_scaled = gru_model(forecast_input_tensor).cpu().numpy().flatten()[0]

                scaler_y = StandardScaler()
        scaler_y.fit(data[['chl_a']].values)
        forecast_value = scaler_y.inverse_transform(np.array([[forecast_value_scaled]])).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

GRU_forecast_values, forecast_dates = forecast_next_records_gru(best_gru_model, data_LSTM, lag=lag, forecast_steps=forecast_steps, device=device)

for date, value in zip(forecast_dates, GRU_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


# Random Forest

### Hyperparameter Tunning

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = gru_out[:, -1, :]          out = self.fc(gru_out)
        return out

def prepare_data_with_time_steps(data_LSTM, time_steps=lag):
    chl_a_values = data_LSTM['chl_a'].values
    month_sin = data_LSTM['month_sin'].values
    month_cos = data_LSTM['month_cos'].values

    X, y = [], []
    for i in range(len(chl_a_values) - time_steps):
        X.append(
               np.hstack([chl_a_values[i:i + time_steps].reshape(-1, 1), 
               np.tile([month_sin[i + time_steps], month_cos[i + time_steps]], (time_steps, 1))])
        )
        y.append(chl_a_values[i + time_steps])

    X, y = np.array(X), np.array(y).reshape(-1, 1)
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X = X.reshape(X.shape[0], -1)      X = scaler_X.fit_transform(X)
    X = X.reshape(X.shape[0], time_steps, -1)  
        y = scaler_y.fit_transform(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, scaler_y

def train_and_evaluate_gru(X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=100, lr=0.01):
    gru_model = GRUModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(gru_model.parameters(), lr=lr)
    
        for epoch in range(num_epochs):
        gru_model.train()
        optimizer.zero_grad()
        output = gru_model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        gru_model.eval()
    with torch.no_grad():
        y_train_pred = gru_model(X_train).cpu().numpy()
        y_test_pred = gru_model(X_test).cpu().numpy()

        train_metrics = {
        'R2': r2_score(y_train.cpu().numpy(), y_train_pred),
        'MAE': mean_absolute_error(y_train.cpu().numpy(), y_train_pred),
        'MSE': mean_squared_error(y_train.cpu().numpy(), y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train.cpu().numpy(), y_train_pred)),
        'R': np.corrcoef(y_train.cpu().numpy().flatten(), y_train_pred.flatten())[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test.cpu().numpy(), y_test_pred),
        'MAE': mean_absolute_error(y_test.cpu().numpy(), y_test_pred),
        'MSE': mean_squared_error(y_test.cpu().numpy(), y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test.cpu().numpy(), y_test_pred)),
        'R': np.corrcoef(y_test.cpu().numpy().flatten(), y_test_pred.flatten())[0, 1]
    }

        return train_metrics, test_metrics, y_train_pred, y_test_pred, gru_model


best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_gru_model = None  
time_steps = lag
input_size = 3  hidden_size = 32
output_size = 1

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_time_steps(data_LSTM, time_steps=time_steps)

for i in range(5):
    print(f"Run {i+1}/20")
    train_metrics, test_metrics, y_train_pred, y_test_pred, gru_model = train_and_evaluate_gru(
        X_train, y_train, X_test, y_test, input_size, hidden_size, output_size, num_epochs=100, lr=0.01
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_gru_model = gru_model  

best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred)
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred)
y_train_actual = scaler_y.inverse_transform(y_train.cpu().numpy())
y_test_actual = scaler_y.inverse_transform(y_test.cpu().numpy())

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {'Actual': y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])
GRU_Train = best_y_train_pred.flatten()
GRU_Test = best_y_test_pred.flatten()

with pd.ExcelWriter('GRU.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'Best_GRU_South_TimeSteps.xlsx'")


### Main Code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



def train_and_evaluate_rf(X_train, y_train, X_test, y_test, n_estimators=50, max_depth=None):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=3, random_state=42)

        rf_model.fit(X_train, y_train)

        y_train_pred = rf_model.predict(X_train)
    y_test_pred = rf_model.predict(X_test)

        train_metrics = {
        'R2': r2_score(y_train, y_train_pred),
        'MAE': mean_absolute_error(y_train, y_train_pred),
        'MSE': mean_squared_error(y_train, y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'R': np.corrcoef(y_train, y_train_pred)[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test, y_test_pred),
        'MAE': mean_absolute_error(y_test, y_test_pred),
        'MSE': mean_squared_error(y_test, y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'R': np.corrcoef(y_test, y_test_pred)[0, 1]
    }

    return rf_model, train_metrics, test_metrics, y_train_pred, y_test_pred

best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_y_train_actual = None
best_y_test_actual = None
best_rf_model = None

lag = lag n_estimators = 100
max_depth = 3

X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_lag(data)

for i in range(20):
    print(f"Run {i + 1}/20")
    
        rf_model, train_metrics, test_metrics, y_train_pred, y_test_pred = train_and_evaluate_rf(
        X_train, y_train, X_test, y_test, n_estimators=n_estimators, max_depth=max_depth
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_y_train_actual = y_train
        best_y_test_actual = y_test
        best_rf_model = rf_model

best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred.reshape(-1, 1)).flatten()
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred.reshape(-1, 1)).flatten()
best_y_train_actual = scaler_y.inverse_transform(best_y_train_actual.reshape(-1, 1)).flatten()
best_y_test_actual = scaler_y.inverse_transform(best_y_test_actual.reshape(-1, 1)).flatten()

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {'Actual': best_y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': best_y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])

RF_Train = best_y_train_pred.flatten()
RF_Test = best_y_test_pred.flatten()

with pd.ExcelWriter('RF.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'Best_RF_South_LagFeatures.xlsx'")


### RF Forecast

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_rf(last_lag_values, forecast_dates, i, lag=lag):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values, month_sin, month_cos])

    return forecast_input

def forecast_next_records_rf(rf_model, data, lag=lag, forecast_steps=1):
        last_lag_values = data[[f'yt-{i}' for i in range(1, lag + 1)]].values[-1]

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input = prepare_forecast_input_rf(last_lag_values, forecast_dates, i, lag=lag)

                forecast_value_scaled = rf_model.predict(forecast_input.reshape(1, -1))[0]

                scaler_y = StandardScaler()
        scaler_y.fit(data[['chl_a']].values)
        forecast_value = scaler_y.inverse_transform([[forecast_value_scaled]]).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates


RF_forecast_values, forecast_dates = forecast_next_records_rf(best_rf_model, data, lag=lag, forecast_steps=forecast_steps)

for date, value in zip(forecast_dates, RF_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


# GPR

### Hyperparameters Tunning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid

def train_and_evaluate_gpr(X_train, y_train, X_test, y_test, kernel):
    gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-5, random_state=42)

        gpr_model.fit(X_train, y_train)

        y_test_pred = gpr_model.predict(X_test)

        r2 = r2_score(y_test, y_test_pred)
    return r2, gpr_model

lag = lag
X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_lag(data)

param_grid = {
    'kernel': [
        1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0),
        1.0 * ExpSineSquared(length_scale=1.0, periodicity=12.0) + WhiteKernel(noise_level=1.0),
        1.0 * RBF(length_scale=2.0) + WhiteKernel(noise_level=1.0),
        1.0 * ExpSineSquared(length_scale=2.0, periodicity=12.0) + WhiteKernel(noise_level=1.0),
        1.0 * RBF(length_scale=1.0) + ExpSineSquared(length_scale=1.0, periodicity=12.0) + WhiteKernel(noise_level=1.0)
    ]
}

best_r2 = -float('inf')
best_params = None
best_gpr_model = None

for params in ParameterGrid(param_grid):
    print(f"Testing kernel: {params['kernel']}")
    r2, gpr_model = train_and_evaluate_gpr(X_train, y_train, X_test, y_test, kernel=params['kernel'])

        if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_gpr_model = gpr_model

print(f"Best kernel: {best_params['kernel']}")
print(f"Best R2 score: {best_r2}")



### Main Code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepare_data_with_lag(data):
    X = data[[f'yt-{i}' for i in range(1, lag + 1)] + ['month_sin', 'month_cos']].values
    y = data['chl_a'].values
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
    return X_train, y_train, X_test, y_test, scaler_y

def train_and_evaluate_gpr(X_train, y_train, X_test, y_test, kernel):
    gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-5, random_state=42)

        gpr_model.fit(X_train, y_train)

        y_train_pred = gpr_model.predict(X_train)
    y_test_pred = gpr_model.predict(X_test)

        train_metrics = {
        'R2': r2_score(y_train, y_train_pred),
        'MAE': mean_absolute_error(y_train, y_train_pred),
        'MSE': mean_squared_error(y_train, y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'R': np.corrcoef(y_train, y_train_pred)[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test, y_test_pred),
        'MAE': mean_absolute_error(y_test, y_test_pred),
        'MSE': mean_squared_error(y_test, y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'R': np.corrcoef(y_test, y_test_pred)[0, 1]
    }

    return gpr_model, train_metrics, test_metrics, y_train_pred, y_test_pred

best_params = {
    'kernel': 1.0 * RBF(length_scale=1.0) + ExpSineSquared(length_scale=1.0, periodicity=12.0) + WhiteKernel(noise_level=1.0)
}

lag = lag
X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_lag(data)

best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_y_train_actual = None
best_y_test_actual = None
best_gpr_model = None

for i in range(1):
    print(f"Run {i + 1}/20")
    
        gpr_model, train_metrics, test_metrics, y_train_pred, y_test_pred = train_and_evaluate_gpr(
        X_train, y_train, X_test, y_test, kernel=best_params['kernel']
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_y_train_actual = y_train
        best_y_test_actual = y_test
        best_gpr_model = gpr_model  
best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred.reshape(-1, 1)).flatten()
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred.reshape(-1, 1)).flatten()
best_y_train_actual = scaler_y.inverse_transform(best_y_train_actual.reshape(-1, 1)).flatten()
best_y_test_actual = scaler_y.inverse_transform(best_y_test_actual.reshape(-1, 1)).flatten()

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {'Actual': best_y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': best_y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])

GPR_Train = best_y_train_pred.flatten()
GPR_Test = best_y_test_pred.flatten()


with pd.ExcelWriter('GPR.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'Best_GPR_South_LagFeatures.xlsx'")


### GPR Forecast

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_gpr(last_lag_values, forecast_dates, i, lag=lag):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values, month_sin, month_cos])

    return forecast_input

def forecast_next_records_gpr(gpr_model, data, lag=lag, forecast_steps=1, scaler_y=None):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input = prepare_forecast_input_gpr(last_lag_values, forecast_dates, i, lag=lag)

                forecast_value_scaled = gpr_model.predict(forecast_input.reshape(1, -1))[0]

                forecast_value = scaler_y.inverse_transform([[forecast_value_scaled]]).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates


GPR_forecast_values, forecast_dates = forecast_next_records_gpr(best_gpr_model, data, lag=lag, forecast_steps=forecast_steps, scaler_y=scaler_y)

for date, value in zip(forecast_dates, GPR_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


# SVR

### Hyperparameters Tunning

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid

def train_and_evaluate_svr(X_train, y_train, X_test, y_test, kernel='rbf', C=1.0, epsilon=0.1):
    svr_model = SVR(kernel=kernel, C=C, epsilon=epsilon)

        svr_model.fit(X_train, y_train)

        y_test_pred = svr_model.predict(X_test)

        r2 = r2_score(y_test, y_test_pred)
    return r2, svr_model


lag = lag
X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_lag(data)

param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1.0, 10.0],
    'epsilon': [0.01, 0.1, 0.2]
}

best_r2 = -float('inf')
best_params = None
best_svr_model = None

for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    r2, svr_model = train_and_evaluate_svr(
        X_train, y_train, X_test, y_test, 
        kernel=params['kernel'], 
        C=params['C'], 
        epsilon=params['epsilon']
    )

        if r2 > best_r2:
        best_r2 = r2
        best_params = params
        best_svr_model = svr_model

print(f"Best parameters: {best_params}")
print(f"Best R2 score: {best_r2}")



### Main Code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepare_data_with_lag(data):
    X = data[[f'yt-{i}' for i in range(1, lag + 1)] + ['month_sin', 'month_cos']].values
    y = data['chl_a'].values
    
        scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    return X_train, y_train, X_test, y_test, scaler_y

def train_and_evaluate_svr(X_train, y_train, X_test, y_test, kernel='rbf', C=1.0, epsilon=0.1):
    svr_model = SVR(kernel=kernel, C=C, epsilon=epsilon)

        svr_model.fit(X_train, y_train)

        y_train_pred = svr_model.predict(X_train)
    y_test_pred = svr_model.predict(X_test)

        train_metrics = {
        'R2': r2_score(y_train, y_train_pred),
        'MAE': mean_absolute_error(y_train, y_train_pred),
        'MSE': mean_squared_error(y_train, y_train_pred),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'R': np.corrcoef(y_train, y_train_pred)[0, 1]
    }

    test_metrics = {
        'R2': r2_score(y_test, y_test_pred),
        'MAE': mean_absolute_error(y_test, y_test_pred),
        'MSE': mean_squared_error(y_test, y_test_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'R': np.corrcoef(y_test, y_test_pred)[0, 1]
    }

    return svr_model, train_metrics, test_metrics, y_train_pred, y_test_pred

best_params = {
    'kernel': 'rbf',      'C': 1.0,             'epsilon': 0.1    }

lag = lag
X_train, y_train, X_test, y_test, scaler_y = prepare_data_with_lag(data)

best_r2 = -float('inf')
best_train_metrics = None
best_test_metrics = None
best_y_train_pred = None
best_y_test_pred = None
best_y_train_actual = None
best_y_test_actual = None
best_svr_model = None  
for i in range(20):
    print(f"Run {i + 1}/20")
    
        svr_model, train_metrics, test_metrics, y_train_pred, y_test_pred = train_and_evaluate_svr(
        X_train, y_train, X_test, y_test, 
        kernel=best_params['kernel'], 
        C=best_params['C'], 
        epsilon=best_params['epsilon']
    )

        if test_metrics['R2'] > best_r2:
        best_r2 = test_metrics['R2']
        best_train_metrics = train_metrics
        best_test_metrics = test_metrics
        best_y_train_pred = y_train_pred
        best_y_test_pred = y_test_pred
        best_y_train_actual = y_train
        best_y_test_actual = y_test
        best_svr_model = svr_model  
best_y_train_pred = scaler_y.inverse_transform(best_y_train_pred.reshape(-1, 1)).flatten()
best_y_test_pred = scaler_y.inverse_transform(best_y_test_pred.reshape(-1, 1)).flatten()
best_y_train_actual = scaler_y.inverse_transform(best_y_train_actual.reshape(-1, 1)).flatten()
best_y_test_actual = scaler_y.inverse_transform(best_y_test_actual.reshape(-1, 1)).flatten()

metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])


best_y_preds_test = {'Actual': best_y_test_actual.flatten(), 'Predicted': best_y_test_pred.flatten()}
best_y_preds_train = {'Actual': best_y_train_actual.flatten(), 'Predicted': best_y_train_pred.flatten()}

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([best_test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([best_train_metrics])], ignore_index=True)

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': best_train_metrics['R2'], 'R': best_train_metrics['R'], 
                  'MSE': best_train_metrics['MSE'], 'RMSE': best_train_metrics['RMSE'], 'MAE': best_train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': best_test_metrics['R2'], 'R': best_test_metrics['R'], 
                  'MSE': best_test_metrics['MSE'], 'RMSE': best_test_metrics['RMSE'], 'MAE': best_test_metrics['MAE']}, index=[1])
])

SVR_Train = best_y_train_pred.flatten()
SVR_Test = best_y_test_pred.flatten()

with pd.ExcelWriter('SVR.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'Best_SVR_South_LagFeatures_Cyclic.xlsx'")


In [ ]:
import numpy as np
import pandas as pd

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_svr(last_lag_values, forecast_dates, i, lag=lag):
        month_sin, month_cos = calculate_month_cyclic_features(forecast_dates[i])

        forecast_input = np.hstack([last_lag_values, month_sin, month_cos])

    return forecast_input

def forecast_next_records_svr(svr_model, data, lag=lag, forecast_steps=1, scaler_y=None):
        if not pd.api.types.is_datetime64_any_dtype(data.index):
        data.index = pd.to_datetime(data.index)

        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_values = []

        for i in range(forecast_steps):
                forecast_input = prepare_forecast_input_svr(last_lag_values, forecast_dates, i, lag=lag)

                forecast_value_scaled = svr_model.predict(forecast_input.reshape(1, -1))[0]

                forecast_value = scaler_y.inverse_transform([[forecast_value_scaled]]).flatten()[0]

                forecast_values.append(forecast_value)

                last_lag_values = np.roll(last_lag_values, -1)          last_lag_values[-1] = forecast_value  
    return forecast_values, forecast_dates

forecast_steps = 6  
SVR_forecast_values, forecast_dates = forecast_next_records_svr(best_svr_model, data, lag=lag, forecast_steps=forecast_steps, scaler_y=scaler_y)

for date, value in zip(forecast_dates, SVR_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


In [ ]:
data_for_forecasting

In [ ]:
'''import numpy as np
import torch
from kan import KAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def custom_r2(y_true, y_pred):
    y_true_mean = np.mean(y_true)
    y_pred_mean = np.mean(y_pred)
    numerator = np.mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    denominator = np.sqrt(np.mean((y_true - y_true_mean) ** 2) * np.mean((y_pred - y_pred_mean) ** 2))
    r = numerator / denominator
    return r ** 2

def load_chla_dataset(data, lag=3):
    target = data['chl_a'].values
    input_data = data[[f'yt-{i}' for i in range(1, lag + 1)] + ['month_sin', 'month_cos']].values

        scaler = StandardScaler()
    input_data = scaler.fit_transform(input_data)

        input_tensor = torch.tensor(input_data, dtype=torch.float32, device=device)
    target_tensor = torch.tensor(target, dtype=torch.float32, device=device).reshape(-1, 1)

        train_data, test_data, train_target, test_target = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

        return {
        'train_input': train_data,
        'train_label': train_target,
        'test_input': test_data,
        'test_label': test_target
    }


def build_and_train_kan_model(dataset):
    KAN_model = KAN(
        width=[dataset['train_input'].shape[1], 7, 1],
        grid=3,
        k=3,
        noise_scale=0.1,
        noise_scale_base=0.1,
        symbolic_enabled=False,
        bias_trainable=False,
        grid_eps=1.0,
        grid_range=[-1, 1],
        sp_trainable=True,
        sb_trainable=True,
        device=device
    )

        def train_mse():
        with torch.no_grad():
            predictions = KAN_model(dataset['train_input'])
            return torch.nn.functional.mse_loss(predictions, dataset['train_label'])

    def test_mse():
        with torch.no_grad():
            predictions = KAN_model(dataset['test_input'])
            return torch.nn.functional.mse_loss(predictions, dataset['test_label'])

        results = KAN_model.train(
        dataset, opt="Adam", device=device, metrics=(train_mse, test_mse),
        steps=100,
        log=1,
        lamb=0.01,
        lamb_l1=0.1,
        lamb_entropy=0.01,
        lamb_coef=1,
        lamb_coefdiff=0.1,
        update_grid=True,
        grid_update_num=10,
        loss_fn=torch.nn.SmoothL1Loss(),
        lr=0.01,
        stop_grid_update_step=50,
        batch=-1,
        small_mag_threshold=1e-16,
        small_reg_factor=0.1,
        sglr_avoid=False
    )

        with torch.no_grad():
        y_pred_train = KAN_model(dataset['train_input']).cpu().numpy().flatten()
        y_pred_test = KAN_model(dataset['test_input']).cpu().numpy().flatten()

        train_metrics = {
        'R2': r2_score(dataset['train_label'].cpu().numpy(), y_pred_train),
        'MAE': mean_absolute_error(dataset['train_label'].cpu().numpy(), y_pred_train),
        'MSE': mean_squared_error(dataset['train_label'].cpu().numpy(), y_pred_train),
        'RMSE': np.sqrt(mean_squared_error(dataset['train_label'].cpu().numpy(), y_pred_train)),
        'R': np.corrcoef(dataset['train_label'].cpu().numpy().flatten(), y_pred_train)[0, 1]
    }

    test_metrics = {
        'R2': r2_score(dataset['test_label'].cpu().numpy(), y_pred_test),
        'MAE': mean_absolute_error(dataset['test_label'].cpu().numpy(), y_pred_test),
        'MSE': mean_squared_error(dataset['test_label'].cpu().numpy(), y_pred_test),
        'RMSE': np.sqrt(mean_squared_error(dataset['test_label'].cpu().numpy(), y_pred_test)),
        'R': np.corrcoef(dataset['test_label'].cpu().numpy().flatten(), y_pred_test)[0, 1]
    }

    return KAN_model, train_metrics, test_metrics, y_pred_train, y_pred_test


metrics_test_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])
metrics_train_df = pd.DataFrame(columns=['R2', 'R', 'MSE', 'RMSE', 'MAE'])

best_y_preds_test = {}
best_y_preds_train = {}


chla_dataset = load_chla_dataset(data, lag=lag)

KAN_model, train_metrics, test_metrics, y_pred_train, y_pred_test = build_and_train_kan_model(chla_dataset)

metrics_test_df = pd.concat([metrics_test_df, pd.DataFrame([test_metrics])], ignore_index=True)
metrics_train_df = pd.concat([metrics_train_df, pd.DataFrame([train_metrics])], ignore_index=True)

best_y_preds_test['Actual'] = chla_dataset['test_label'].cpu().numpy().flatten()
best_y_preds_test['Predicted'] = y_pred_test

best_y_preds_train['Actual'] = chla_dataset['train_label'].cpu().numpy().flatten()
best_y_preds_train['Predicted'] = y_pred_train

summary_df = pd.concat([
    pd.DataFrame({'Data': 'Train', 'R2': train_metrics['R2'], 'R': train_metrics['R'], 
                  'MSE': train_metrics['MSE'], 'RMSE': train_metrics['RMSE'], 'MAE': train_metrics['MAE']}, index=[0]),
    pd.DataFrame({'Data': 'Test', 'R2': test_metrics['R2'], 'R': test_metrics['R'], 
                  'MSE': test_metrics['MSE'], 'RMSE': test_metrics['RMSE'], 'MAE': test_metrics['MAE']}, index=[1])
])

with pd.ExcelWriter('KAN.xlsx') as writer:
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    pd.DataFrame(best_y_preds_test).to_excel(writer, sheet_name='Test Predictions', index=False)
    pd.DataFrame(best_y_preds_train).to_excel(writer, sheet_name='Train Predictions', index=False)
    summary_df.to_excel(writer, sheet_name='Metrics Summary', index=False)

print("Process completed and results saved to 'KAN_South.xlsx'")
'''

# TEST  DO NOT USE

In [ ]:
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tkan import TKAN
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

BACKEND = 'jax'
os.environ['KERAS_BACKEND'] = BACKEND

keras.utils.set_random_seed(1)

N_MAX_EPOCHS = 100
BATCH_SIZE = 24

early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
)
lr_callback = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    min_lr=0.00025,
)

callbacks = [early_stopping_callback, lr_callback]

def generate_data(df, sequence_length, n_ahead=1):
    X = df[[f'yt-{i}' for i in range(1, sequence_length + 1)] + ['month_sin', 'month_cos']].values
    y = df['chl_a'].values

        X = X.reshape((X.shape[0], sequence_length, -1))

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, X_test, y_train, y_test

sequence_length = 1  X_train, X_test, y_train, y_test = generate_data(data_LSTM, sequence_length, 1)

TKAN_model = Sequential([
    Input(shape=(sequence_length, X_train.shape[2])),
    TKAN(100, sub_kan_configs=[{'spline_order': 3, 'grid_size': 5}], return_sequences=False),
            Dense(units=1, activation='linear')  ])

TKAN_model.compile(optimizer="Adam", loss='mean_squared_error')

history = TKAN_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, 
                          validation_split=0.2, shuffle=False)

y_train_pred = TKAN_model.predict(X_train).flatten()
y_test_pred = TKAN_model.predict(X_test).flatten()

def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100      r2 = r2_score(y_true, y_pred)
    r, p_value = pearsonr(y_true, y_pred)
    return mse, rmse, mae, mape, r2, r, p_value

train_metrics = calculate_metrics(y_train, y_train_pred)
test_metrics = calculate_metrics(y_test, y_test_pred)

train_results_df = pd.DataFrame({
    'Actual Train': y_train.flatten(),
    'Predicted Train': y_train_pred
})

test_results_df = pd.DataFrame({
    'Actual Test': y_test.flatten(),
    'Predicted Test': y_test_pred
})

def save_to_excel(file_name, train_df, test_df, metrics_df):
    with pd.ExcelWriter(file_name) as writer:
        train_df.to_excel(writer, sheet_name='Train Results', index=False)
        test_df.to_excel(writer, sheet_name='Test Results', index=False)
        metrics_df.to_excel(writer, sheet_name='Metrics', index=False)
    print(f"Exported actual vs predicted values and metrics to {file_name}")

TKAN_Train = y_train_pred
TKAN_Test = y_test_pred

metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'RMSE', 'MAE', 'MAPE', 'R²', 'R', 'P-value'],
    'Training': train_metrics,
    'Testing': test_metrics
})

save_to_excel("TKAN.xlsx", train_results_df, test_results_df, metrics_df)


In [ ]:
TKAN_model.summary()

In [ ]:
import numpy as np
import pandas as pd

def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    return month_sin, month_cos

def prepare_forecast_input_tkan(last_lag_values, forecast_dates, lag=lag):
        forecast_inputs = []
    for date in forecast_dates:
                month_sin, month_cos = calculate_month_cyclic_features(date)
        
                forecast_input = np.hstack([last_lag_values.reshape(-1, 1), np.tile([month_sin, month_cos], (lag, 1))])
        forecast_inputs.append(forecast_input)
    
        forecast_inputs = np.array(forecast_inputs)
    return forecast_inputs

def forecast_next_records_tkan(TKAN_model, data, lag=lag, forecast_steps=12):
        last_lag_values = data['chl_a'].values[-lag:].flatten()

        last_date = data.index[-1]
    forecast_dates = [last_date + pd.DateOffset(months=i) for i in range(1, forecast_steps + 1)]

        forecast_input = prepare_forecast_input_tkan(last_lag_values, forecast_dates, lag)

        forecast_values_scaled = TKAN_model.predict(forecast_input)

        scaler_y = StandardScaler()
    scaler_y.fit(data[['chl_a']].values)
    forecast_values = scaler_y.inverse_transform(forecast_values_scaled)

    return forecast_values.flatten(), forecast_dates

forecast_steps = 12  lag = lag  
TKAN_forecast_values, forecast_dates = forecast_next_records_tkan(TKAN_model, data_LSTM, lag=lag, forecast_steps=forecast_steps)

for date, value in zip(forecast_dates, TKAN_forecast_values):
    print(f"Forecasted Chl-a concentration for {date.date()}: {value:.4f}")


In [ ]:
import pandas as pd


forecast_data = {
    'Date': forecast_dates,      'Actual': data_for_forecasting['chl_a'].values,      'MLP_Forecast': ANN_forecast_values,
    'RF_Forecast': RF_forecast_values,
    'LSTM_Forecast': LSTM_forecast_values,
    'GRU_Forecast': GRU_forecast_values,
    'GPR_Forecast': GPR_forecast_values,
    'SVR_Forecast': SVR_forecast_values
}

forecast_df = pd.DataFrame(forecast_data)

forecast_df.to_excel('forecast.xlsx', index=False)

print("Forecast results have been successfully exported to 'forecast.xlsx'")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm

def calculate_metrics(actual, predicted):
    r2 = r2_score(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = mean_squared_error(actual, predicted, squared=False)
    mae = mean_absolute_error(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100 if np.any(actual) else 0
    X = sm.add_constant(predicted)
    model = sm.OLS(actual, X).fit()
    p_value = model.pvalues[1]
    return r2, mse, rmse, mae, mape, p_value

train_df = pd.DataFrame({
    'Actual': Act_Train,
    'ANN': ANN_Train,
    'LSTM': LSTM_Train,
    'GRU': GRU_Train,
    'RF': RF_Train, 
    'GPR': GPR_Train,
    'SVR': SVR_Train
})

test_df = pd.DataFrame({
    'Actual': Act_Test,
    'ANN': ANN_Test,
    'LSTM': LSTM_Test,
    'GRU': GRU_Test,
    'RF': RF_Test, 
    'GPR': GPR_Test,
    'SVR': SVR_Test
})

models = ['ANN', 'LSTM', 'GRU', 'RF', 'GPR', 'SVR']

def calculate_all_metrics(actual_train, actual_test):
    metrics_train = {'Model': [], 'R2': [], 'MSE': [], 'RMSE': [], 'MAE': [], 'MAPE': [], 'p-value': []}
    metrics_test = {'Model': [], 'R2': [], 'MSE': [], 'RMSE': [], 'MAE': [], 'MAPE': [], 'p-value': []}
    
    for model in models:
                preds_train = globals()[f"{model}_Train"]
        metrics_train['Model'].append(model)
        metrics_train_values = calculate_metrics(actual_train, preds_train)
        metrics_train['R2'].append(metrics_train_values[0])
        metrics_train['MSE'].append(metrics_train_values[1])
        metrics_train['RMSE'].append(metrics_train_values[2])
        metrics_train['MAE'].append(metrics_train_values[3])
        metrics_train['MAPE'].append(metrics_train_values[4])
        metrics_train['p-value'].append(metrics_train_values[5])

                preds_test = globals()[f"{model}_Test"]
        metrics_test['Model'].append(model)
        metrics_test_values = calculate_metrics(actual_test, preds_test)
        metrics_test['R2'].append(metrics_test_values[0])
        metrics_test['MSE'].append(metrics_test_values[1])
        metrics_test['RMSE'].append(metrics_test_values[2])
        metrics_test['MAE'].append(metrics_test_values[3])
        metrics_test['MAPE'].append(metrics_test_values[4])
        metrics_test['p-value'].append(metrics_test_values[5])
    
    return pd.DataFrame(metrics_train), pd.DataFrame(metrics_test)

metrics_train_df, metrics_test_df = calculate_all_metrics(Act_Train, Act_Test)

with pd.ExcelWriter('model_predictions_metrics.xlsx') as writer:
    train_df.to_excel(writer, sheet_name='Train Actual vs Predicted', index=False)
    test_df.to_excel(writer, sheet_name='Test Actual vs Predicted', index=False)
    metrics_train_df.to_excel(writer, sheet_name='Train Metrics', index=False)
    metrics_test_df.to_excel(writer, sheet_name='Test Metrics', index=False)

print("Excel file saved with model predictions and metrics.")


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Input

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from tkan import TKAN  
import time

keras.utils.set_random_seed(1)

N_MAX_EPOCHS = 1000
BATCH_SIZE = 24

early_stopping_callback = lambda: keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=10,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda: keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda: [
    early_stopping_callback(),
    lr_callback(),
    keras.callbacks.TerminateOnNaN(),
]


X = data[[f'yt-{i}' for i in range(1, lag + 1)] + ['month_sin', 'month_cos']].values
y = data['chl_a'].values

def create_sequences(X, y, sequence_length):
    Xs, ys = [], []
    for i in range(len(X) - sequence_length):
        Xs.append(X[i : (i + sequence_length)])
        ys.append(y[i + sequence_length])
    return np.array(Xs), np.array(ys)

sequence_length = 12  
X_seq, y_seq = create_sequences(X, y, sequence_length)

train_size = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]

scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(
    X_train.shape
)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[2])).reshape(
    X_test.shape
)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

def build_model(model_type, input_shape):
    if model_type == 'TKAN':
        model = Sequential(
            [
                Input(shape=input_shape),
                TKAN(32, return_sequences=True),
                TKAN(32, sub_kan_output_dim = 15, sub_kan_input_dim = 10, return_sequences=False),
                Dense(units=1, activation='linear'),
            ],
            name=model_type,
        )
    elif model_type == 'GRU':
        model = Sequential(
            [
                Input(shape=input_shape),
                GRU(100, return_sequences=True),
                GRU(100, return_sequences=False),
                Dense(units=1, activation='relu'),
            ],
            name=model_type,
        )
    elif model_type == 'LSTM':
        model = Sequential(
            [
                Input(shape=input_shape),
                LSTM(32, return_sequences=True),
                LSTM(64, return_sequences=False),
                Dense(units=1,activation=None),
            ],
            name=model_type,
        )
    else:
        raise ValueError(f"Unknown model_type {model_type}")
    return model

models = ['TKAN', 'GRU', 'LSTM'] 
train_predictions = {}
test_predictions = {}
train_actuals = None
test_actuals = None

for model_type in models:
    print(f"\nTraining {model_type} model...")
    model = build_model(model_type, X_train_scaled.shape[1:])
    optimizer = keras.optimizers.Adam(0.001)
    model.compile(
        optimizer=optimizer, loss='mean_squared_error', jit_compile=True
    )
    model.summary()

        start_time = time.time()
    history = model.fit(
        X_train_scaled,
        y_train_scaled,
        validation_split=0.2,
        epochs=N_MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks(),
                verbose=1,
    )
    end_time = time.time()
    training_time = end_time - start_time

        y_train_pred_scaled = model.predict(X_train_scaled)
    y_test_pred_scaled = model.predict(X_test_scaled)

        y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled)
    y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

        train_predictions[model_type] = y_train_pred
    test_predictions[model_type] = y_test_pred

        if train_actuals is None:
        y_train_actual = scaler_y.inverse_transform(y_train_scaled)
        y_test_actual = scaler_y.inverse_transform(y_test_scaled)
        train_actuals = y_train_actual
        test_actuals = y_test_actual

    keras.backend.clear_session()

import scipy.stats as stats

train_results_df = pd.DataFrame({'Actual': train_actuals.flatten()})
test_results_df = pd.DataFrame({'Actual': test_actuals.flatten()})

train_metrics = {}
test_metrics = {}

for model_name in models:
    y_train_pred = train_predictions[model_name]
    y_test_pred = test_predictions[model_name]

        train_results_df[model_name] = y_train_pred.flatten()
    test_results_df[model_name] = y_test_pred.flatten()

        train_r2 = r2_score(train_actuals, y_train_pred)
    train_mse = mean_squared_error(train_actuals, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    train_mae = np.mean(np.abs(train_actuals - y_train_pred))
    train_mape = np.mean(np.abs((train_actuals - y_train_pred) / train_actuals)) * 100
    train_r, train_p_value = stats.pearsonr(train_actuals.flatten(), y_train_pred.flatten())

        train_metrics[model_name] = {
        'R2': train_r2,
        'MSE': train_mse,
        'RMSE': train_rmse,
        'MAE': train_mae,
        'MAPE': train_mape,
        'R': train_r,
        'p-value': train_p_value
    }

        test_r2 = r2_score(test_actuals, y_test_pred)
    test_mse = mean_squared_error(test_actuals, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    test_mae = np.mean(np.abs(test_actuals - y_test_pred))
    test_mape = np.mean(np.abs((test_actuals - y_test_pred) / test_actuals)) * 100
    test_r, test_p_value = stats.pearsonr(test_actuals.flatten(), y_test_pred.flatten())

        test_metrics[model_name] = {
        'R2': test_r2,
        'MSE': test_mse,
        'RMSE': test_rmse,
        'MAE': test_mae,
        'MAPE': test_mape,
        'R': test_r,
        'p-value': test_p_value
    }

train_metrics_df = pd.DataFrame(train_metrics).T
test_metrics_df = pd.DataFrame(test_metrics).T

with pd.ExcelWriter('Model_Results.xlsx') as writer:
    train_results_df.to_excel(writer, sheet_name='Train Results', index=False)
    test_results_df.to_excel(writer, sheet_name='Test Results', index=False)
    train_metrics_df.to_excel(writer, sheet_name='Train Metrics')
    test_metrics_df.to_excel(writer, sheet_name='Test Metrics')

In [ ]:
def calculate_month_cyclic_features(date):
    month = date.month
    month_sin = np.sin(2 * np.pi * (month - 1) / 12)
    month_cos = np.cos(2 * np.pi * (month - 1) / 12)
    return month_sin, month_cos

forecast_steps = 12
lag = lag  
last_sequence = data_LSTM[['chl_a', 'month_sin', 'month_cos']].values[-lag:]

forecast_dates = pd.date_range(
    start=data.index[-1] + pd.DateOffset(months=1),      periods=forecast_steps,
    freq='MS'
)

forecast_df = pd.DataFrame({'Date': forecast_dates})

for model_name in models:
    print(f"\nForecasting with {model_name} model...")
    forecasts = []
    last_sequence_copy = last_sequence.copy()

        model = build_model(model_name, X_train_scaled.shape[1:])

        for i in range(forecast_steps):
                print(last_sequence_copy)

                forecast_date = forecast_dates[i]

                month_sin, month_cos = calculate_month_cyclic_features(forecast_date)

                                new_input = last_sequence_copy.copy()

                        new_input_full = np.zeros((lag, 14))          new_input_full[:, 0:3] = new_input  
                new_input_full[-1, 1] = month_sin          new_input_full[-1, 2] = month_cos  
                new_input_scaled = scaler_X.transform(new_input_full)
        X_input = new_input_scaled.reshape(1, *new_input_scaled.shape)  
                y_forecast_scaled = model.predict(X_input)
        y_forecast = scaler_y.inverse_transform(y_forecast_scaled).flatten()[0]

                forecasts.append(y_forecast)

                        new_row = np.zeros(14)          new_row[0] = y_forecast          new_row[1] = month_sin           new_row[2] = month_cos           
                last_sequence_copy = np.vstack([new_input_full[1:], new_row])

        forecast_df[model_name] = forecasts

with pd.ExcelWriter('Model_Results.xlsx', mode='a', engine='openpyxl') as writer:
    forecast_df.to_excel(writer, sheet_name='Forecasts', index=False)


In [ ]:
data_LSTM